# Why I like Julia
by Victor Boussange

## Julia is expressive and understandable
- In e.g. Python or R, libraries are mostly written in non-native language. [Here is an example with `pytorch`](https://github.com/pytorch/pytorch).

<p align="center">
  <img src="img/pytorch.png" />
</p>

- Source code of most Julia libraries is written in **pure Julia**. [Here is an example with Flux.jl](https://github.com/FluxML/Flux.jl), the equivalent of `pytorch` in Julia.


<p align="center">
  <img src="img/flux.png" />
</p>



- This allows to understand exactly what the functions you are using are doing ➡️ this makes you smarter

[Here is an example of how is implemented a dense layer in Julia](https://github.com/FluxML/Flux.jl/blob/0038a60e266d0fca17aa8db99cd6453eb633ee7b/src/layers/basic.jl#L170)

```julia
function (a::Dense)(x::AbstractVecOrMat)
  _size_check(a, x, 1 => size(a.weight, 2))
  σ = NNlib.fast_act(a.σ, x)  # replaces tanh => tanh_fast, etc
  xT = _match_eltype(a, x)  # fixes Float64 input, etc.
  return σ.(a.weight * xT .+ a.bias)
end
```

- This allows you to easily contribute and develop to packages ➡️ this makes you more useful in this world




- The language syntax is very close to mathematical symbolics ➡️ benefits **understandability**

![](https://github.com/mossr/BeautifulAlgorithms.jl/raw/master/img/png/loss_functions.png)

- The language is **emoji friendly**! This is how you would define a Lotka Volterra system in Julia

In [1]:
function lotka_volterra!(du, u, p, t)
    α, β, γ, δ = p
    🐰, 🦊 = u
    d🐰, d🦊 = du
    d🐰 = α * 🐰 - β * 🦊 * 🐰
    d🦊 = γ*🐰 * 🦊  - δ * 🦊
end

lotka_volterra! (generic function with 1 method)

## Julia is interactive

- Amazing IDE with VS code and inline prompts

![](https://code.visualstudio.com/assets/docs/languages/julia/overview.png)

- Julia code be **developed interactively** on a remote server. More on that later on.


## Execution speed!

Here is a benchmark of different tasks performed with commonly used scientific languages.

![](https://julialang.org/assets/images/benchmarks.svg)

Here is another benchmark where the task consists of solving a differential equation model

![](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/figures/wrapper_packages_2_1.png)

#### Resources
- [Julia micro-benchmarks](https://julialang.org/benchmarks/)
- [SciML benchmarks](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/ode_wrapper_packages/)

### Support for parallelism


```julia
Threads.@threads for i in 1:n
    myarray[1] = do_stuff(parameters[i])
end
```

## Community well organized

There is an active community of users and developers who contribute high-quality packages regularly.

- Slack channel
- Discourse forum
- Youtube tutorials

Information available at https://julialang.org/community/

## Multiple Dispatch

Julia's multiple dispatch mechanism allows **functions to be specialized for different types of data**

Thanks to multiple dispatch, code can be effortlessly made **generic** but also very **specific** (without thinking of class inheritance etc, as you would do in Python)

Multiple dispatch explained to biologists

In [4]:
abstract type PlantSpecies end

struct Oak <: PlantSpecies
    height::Float64
    leaf_area::Float64
end

struct Maple <: PlantSpecies
    height::Float64
    leaf_area::Float64
end

function aboveground_biomass(species::Oak)
    return 0.0314 * species.height^2.19 * species.leaf_area^0.91
end

function aboveground_biomass(species::Maple)
    return 0.0215 * species.height^2.42 * species.leaf_area^0.94
end


aboveground_biomass (generic function with 2 methods)

### Interesting use case of multiple dispatch: GPU computing

Multiple dispatch can be useful when it comes to GPU computing because it allows for efficient dispatch of functions to the GPU, which can lead to significant performance gains. This is achieved through Julia's `GPUArrays` package, which provides a GPU-backed array type that can be used in place of regular Julia arrays.

When a function is called on a `GPUArray`, Julia's multiple dispatch mechanism can determine if a GPU version of the function is available, and if so, dispatch the computation to the GPU. This allows computations to be executed in parallel on the GPU, which can lead to significant performance gains over executing the same computation on the CPU.

Here's an example:

```julia
using CUDA

function add_matrices(a::AbstractArray, b::AbstractArray)
    return a + b
end

# generate CPU arrays
a = rand(1000, 1000)
b = rand(1000, 1000)

# call the function on CPU arrays
c = add_matrices(a, b)

# generate GPU arrays
a_gpu = CUDA.rand(1000, 1000)
b_gpu = CUDA.rand(1000, 1000)

# call the function on GPU arrays
c_gpu = add_matrices(a_gpu, b_gpu)

```



In this example, we define a function `add_matrices` that adds two arrays together. When called with `a_gpu` and `b_gpu`, Julia's multiple dispatch mechanism recognizes that a GPU version of the function is available and dispatches the computation to the GPU.

The resulting `c_gpu` array contains the result of the computation, which can be copied back to the CPU using the `Array` function.

In summary, multiple dispatch in Julia can be useful when it comes to GPU computing because it allows functions to be specialized for different types of data and to dispatch computations to the GPU when appropriate. This can lead to significant performance gains, especially when working with large arrays.

## Julia integrates well with existing code

Calling C, Fortran, Python, and other libraries from  Julia is easy thanks to its first class support for interoperability.

More on that later on.

![](https://github.com/JuliaInterop/RCall.jl/raw/master/ggplot.png)

## Composability of libraries and scientific ML
### Julia is an automatic differentiation pervasive language [(Innes et al., 2019)](http://arxiv.org/abs/1907.07587)

### The [SciML](https://sciml.ai) ecosystem

SciML is a composable open source software for scientific machine learning with differentiable programming.
   -  Read [this very cool paper](https://arxiv.org/abs/2001.04385) introducing the [SciML software ecosystem]
   - [Example using Deep Learning libraries in combination with ODE solvers](ode_solvers_with_DL.jl)

```julia
function lotka_volterra(u, p, t)
    weights_nn, α, δ = p
    🐰, 🦊 = u

    nn_res = neuralnet(🦊, 🐰, weights_nn)

    d🐰 = α * 🐰 - nn_res[1]
    d🦊 = nn_res[2]  - δ * 🦊

    return [d🐰, d🦊]
end
```

### Bayesian inference framework with deep learning

[Here is another cool example](https://turinglang.org/v0.24/tutorials/03-bayesian-neural-network/) of composability between [`Turing.jl`](https://turinglang.org/v0.24/) and `Flux.jl`

![](https://turinglang.org/v0.24/tutorials/figures/03_bayesian-neural-network_9_1.png)

## Package management

Julia provides a **built-in package** manager called `Pkg` for **managing packages and environments**. Users can create a new environment and add specific packages to it, and each environment has its own set of dependencies. Julia also allows users to switch between different environments easily.

<div align="center"><img src="https://pkgdocs.julialang.org/v1/assets/logo.png" width="400"></img> </div>

More on that later on!

## Productivity

- Research script can be easily transformed into **packages**, directly available to the community

## Caveat: trial-and-error development process

Julia is still in a trial and development process. You may expect some bugs in the libraries from time to time, or some rapid changes in syntax.

## A few packages that I have developed during my PhD

### [EvoId.jl](https://github.com/vboussange/EvoId.jl)
Evolutionary Individual based modelling, mathematically grounded. A user friendly package aimed at simulating the evolutionary dynamics of a population structured over a complex spatio-evolutionary structures.

<div align="center"><img src="https://github.com/vboussange/EvoId.jl/raw/master/docs/src/assets/gif_evoid.gif" width="400"></img> </div>

### [HighDimPDE.jl](https://github.com/SciML/HighDimPDE.jl)
Solver for **highly dimensional, non-local, nonlinear PDEs** of the form

$$
\begin{aligned}
    (\partial_t u)(t,x) &= \int_{\Omega} f\big(t,x,{\bf x}, u(t,x),u(t,{\bf x}), ( \nabla_x u )(t,x ),( \nabla_x u )(t,{\bf x} ) \big) d{\bf x} \\
    & \quad + \big\langle \mu(t,x), ( \nabla_x u )( t,x ) \big\rangle + \tfrac{1}{2} \text{Trace} \big(\sigma(t,x) [ \sigma(t,x) ]^* ( \text{Hess}_x u)(t, x ) \big).
\end{aligned}
$$

where $u \colon [0,T] \times \Omega \to \mathbb{R}, \Omega \subseteq \mathbb{R}^{d}$ is subject to initial and boundary conditions, and where $d$ is large.

It is now part of the SciML ecosystem.

### [PiecewiseInference.jl](https://github.com/vboussange/PiecewiseInference.jl)
Suite for parameter inference and model selection with dynamical models characterised by complex dynamics.
<div align="center"><img src="https://github.com/vboussange/PiecewiseInference.jl/raw/main/docs/animated.gif
" width="400"></img> </div>

### [ParametricModels.jl](https://github.com/vboussange/ParametricModels.jl)
Utilities for parametric and composite differential equation models.

### [EcoEvoModelZoo.jl](https://github.com/vboussange/EcoEvoModelZoo.jl)
A zoo of eco-evolutionary models with high fitness.

<div align="center"><img src="https://github.com/vboussange/EcoEvoModelZoo.jl/raw/main/docs/src/time_series_5_species_ecosyste_model.png" width="800"></img> </div>

## Acknowledgements
- This has been partly inspired from [juliazoid blog post](https://juliazoid.com/10-simple-reasons-to-learn-the-julia-programming-language-in-2023-453a4dafab32)